In [22]:
from bs4 import BeautifulSoup
from requests import get
import random
import string
import sqlite3
import logging

# Input
reviewschview_file = '../Scraping/reviewschview.txt'
current_reviews = set(line.strip() for line in open(reviewschview_file))

# Output
log_path = '../Logs/movieReviews.log'
db_path = '../Data/test.db'
db_name = 'test'
conn = sqlite3.connect(db_path)
c = conn.cursor()

logger = logging.getLogger()
fhandler = logging.FileHandler(filename=log_path, mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

def generateNode(length):
    letters_and_digits = string.ascii_letters + string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    node = "_:znode" + result_str
    return node

# all moviereviews
# soup = BeautifulSoup(get("http://reviewschview.com/movies/a-z/page/1").text, 'html.parser')
# movie_container = soup.find_all("p", class_="movie-rating-count")
movie_count = 0
review_count = 0
# for item in movie_container:
#     review_link = item.a['href']
#     if (review_link in current_reviews):
#         pass
#     else:
review_link = 'http://reviewschview.com/dulha-mil-gaya/'
movie_count += 1
review_soup = BeautifulSoup(get(review_link).text, 'lxml')
reviews = review_soup.find_all("li", itemprop="review")
# print(reviews[5])
# print(reviews)
try:
    for item in reviews:
        reviewBody = item.find("div", itemprop="reviewBody").text
        # try:
        worstRating = item.find("meta", itemprop="worstRating")["content"]
        bestRating = item.find("meta", itemprop="bestRating")["content"]
        # except:
        #     worstRating ="0"
        #     bestRating ="99"
        ratingValue = item.find("p", itemprop="ratingValue").text
        # ratingValue = item.find("p", class_="critic-rating-count")
        reviewRating = "already included"
        node = generateNode(31)
        # print(item)

        print('node:' + node)
        print('url: '+ review_link)
        print('reviewBody: ' + reviewBody)
        print('worstRating: ' + worstRating)
        print('bestRating: ' + bestRating)
        print('ratingValue: ' + ratingValue)

        review_count += 1
except Exception as e:
    print("Error " + str(review_link))
    raise(e)
logging.debug('open movies to get reviews from: ' + str(movie_count))
logging.debug('amount of reviews extracted: ' + str(review_count))





node:_:znodeysSgX0Mkyjiky6FEeqNd6NLqXSG24UR
url: http://reviewschview.com/dulha-mil-gaya/
reviewBody: 
The film has a predictable, even clichéd, storyline but writer-director Mudassar Aziz makes a half-decent watch out of it.

worstRating: 0
bestRating: 99
ratingValue: 49
node:_:znodePzXzOubA5BYz4koyx9kwYiR0HtCqa72
url: http://reviewschview.com/dulha-mil-gaya/
reviewBody: 
...works in bits and pieces only and fills you with a sense of wasted opportunity, despite the overt shimmer.

worstRating: 0
bestRating: 99
ratingValue: 49
node:_:znodetEDYBG0650lDBinfEetsLKOAvY4kVyd
url: http://reviewschview.com/dulha-mil-gaya/
reviewBody: 
...regressive and stale.

worstRating: 0
bestRating: 99
ratingValue: 39
node:_:znodeA4yVnflTtUkx9ZHDraO30Q83paLGq2K
url: http://reviewschview.com/dulha-mil-gaya/
reviewBody: 
...hotch potch of a screenplay and unwitty dialogues along with bad direction...

worstRating: 0
bestRating: 99
ratingValue: 39
node:_:znodeC75ejznNbhpVM3b53iaZyi6xvhFiU6W
url: http://revie